## Sentiment_Analysis

In [1]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


## Dataset

In [2]:
!kaggle datasets download -d abhi8923shriv/sentiment-analysis-dataset

Dataset URL: https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset
License(s): CC0-1.0
100% 54.4M/54.4M [00:01<00:00, 39.5MB/s]
100% 54.4M/54.4M [00:01<00:00, 28.8MB/s]


In [3]:
import zipfile

In [4]:
zip=zipfile.ZipFile('/content/sentiment-analysis-dataset.zip','r')
zip.extractall('/content')
zip.close()

In [5]:
import pandas as pd
data=pd.read_csv('/content/train.csv',encoding='ISO-8859-1')

In [6]:
data

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26
...,...,...,...,...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,night,31-45,Ghana,31072940,227540.0,137
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,morning,46-60,Greece,10423054,128900.0,81
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,noon,60-70,Grenada,112523,340.0,331
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,night,70-100,Guatemala,17915568,107160.0,167


## Splitting Data

In [7]:
x=data['text']

In [8]:
x

,text
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
...,...
27476,wish we could come see u on Denver husband l...
27477,I`ve wondered about rake to. The client has ...
27478,Yay good for both of you. Enjoy the break - y...
27479,But it was worth it ****.


In [9]:
y=data['sentiment']

In [10]:
y

,sentiment
0,neutral
1,negative
2,negative
3,negative
4,negative
...,...
27476,negative
27477,negative
27478,positive
27479,positive


## Preprocessing

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
x = [str(text) for text in x if text is not None]

In [13]:
def preprocess_text(text):
    text = text.replace(',', ' ')
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'_', ' ', text)
    word_tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_text = [word for word in word_tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_text]
    return ' '.join(lemmatized_text)
x = [preprocess_text(text) for text in x]

In [14]:
tf=Tokenizer()
tf.fit_on_texts(x)

In [15]:
x=tf.texts_to_sequences(x)

In [16]:
len(tf.word_index)

25565

In [17]:
x=pad_sequences(x,padding='post')

In [18]:
len(x[0])

25

In [19]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y= to_categorical(y)

In [20]:
y

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [25]:

from sklearn.model_selection import train_test_split

X_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)


## Building Model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding

## Using Keras Tuner Library to find the best model

In [23]:
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam


def build_model(hp):
    model = Sequential()
    model.add(Embedding(
        input_dim=len(tf.word_index)+1,
        output_dim=hp.Int('output_dim', min_value=50, max_value=200, step=50),
        input_length=35
    ))

    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32)))

    model.add(Dense(3, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model
import tensorflow as tf1
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=4,
    hyperband_iterations=1,
    directory='dir',
    project_name='Sentiment'
)

early_stop = tf1.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

tuner.search(x, y, validation_split=0.1, epochs=4, callbacks=[early_stop])

model = tuner.get_best_models(num_models=1)[0]


Trial 10 Complete [00h 02m 20s]
val_accuracy: 0.6918879747390747

Best val_accuracy So Far: 0.7020734548568726
Total elapsed time: 00h 25m 22s


## Accuracy Report

In [26]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

model = tuner.get_best_models(num_models=1)[0]
y_val_pred = model.predict(x_val)

y_val_pred_classes = np.argmax(y_val_pred, axis=1)
y_val_true_classes = np.argmax(y_val, axis=1)

accuracy = accuracy_score(y_val_true_classes, y_val_pred_classes)
print(f'Accuracy: {accuracy}')

f1 = f1_score(y_val_true_classes, y_val_pred_classes, average='weighted')
print(f'F1 Score: {f1}')

print("\nClassification Report:")
print(classification_report(y_val_true_classes, y_val_pred_classes))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Accuracy: 0.7997089321448062
F1 Score: 0.7996152967352523

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.71      0.79      1562
           1       0.74      0.83      0.78      2230
           2       0.82      0.84      0.83      1705

    accuracy                           0.80      5497
   macro avg       0.81      0.79      0.80      5497
weighted avg       0.81      0.80      0.80      5497

